# Integrating with Standard Python
In this chapter, you'll learn how to make sure that objects that store the same data are considered equal, how to define and customize string representations of objects, and even how to create new error types. Through interactive exercises, you’ll learn how to further customize your classes to make them work more like standard Python data types.

## Operating overloading: comparison <a name="one"></a>
### Object equality

        class Customer:
            def __init__(self, name, balance):
                self.name, self.balance = name, balance


        customer1 = Customer("Maryam Azar", 3000)
        customer2 = Customer("Maryam Azar", 3000)
        customer1 == customer2                       # <--- returns False

- This situation might make sense: we can have 2 customers with same name and balance
- But what if each customer has a unique ID?
- The two customers should be treated equal but they aren't

        class Customer:
            def __init__(self, name, balance, id):
                self.name, self.balance = name, balance
                self.id = id


        customer1 = Customer("Maryam Azar", 3000, 123)
        customer2 = Customer("Maryam Azar", 3000, 123)
        customer1 == customer2                       # <--- returns False

- The reason Python doesn't consider two object with same data equal by default has to do with how the objects and variables representing them are stored

### Variables and references

![Ref](imgs/references.png)

- When an object is created, Python allocates a chunk of memory to that object
- The variable the object is assigned to contains just the reference to the memory chunk
- When we compare variables `customer1` and `customer2`, we are comparing references not the data
- Since `customer1` and `customer2` point to different chunks of memory, they aren't considered equal

### Custom Comparison
- Doesn't have to be that way

                import numpy as np

                # Two different arrays containing the same data
                array1 = np.array([1,2,3])
                array2 = np.array([1,2,3])

                array1 == array2              # <--- returns True


- Python considers two arrays with same data as equal
- Same with `pandas` dataframes and many other objects
- How can we enforce this for our custom classes?

### Overloading `__eq__()`
- `__eq__()` is called when 2 objects of same class are compared using `==`
- accepts 2 arguments, `self` and `other`, referring to the objects to compare
- returns a Boolean

                class Customer:
                        def __init__(self, id, name):
                                self.id, self.name = id, name
                        # Will be called when == is used
                        def __eq__(self, other):
                                # Diagnostic printout
                                print("__eq__() is called!")

                                # Returns True if all attributes match
                                return (self.id == other.id) and (self.name == other.name)

### Comparison of objects

                # Two equal objects
                customer1 = Customer(123, "Maryam Azar")
                customer2 = Customer(123, "Maryam Azar")

                customer1 == customer2                    # <--- returns True

                # Two unequal objects - different ids
                customer1 = Customer(123, "Maryam Azar")
                customer2 = Customer(456, "Maryam Azar")

                customer1 == customer2                    # <--- returns True

### Other comparison operators

![Comparison operators](imgs/comparison.png)

- `__hash__()` to use objects as dictionary keys and in sets

## Operator overloading: string representation <a name="two"></a>
### Printing an object
- Printing an object of a custom class returns the object's address in memory by default
- For other classes the printout is much more informative

![Print](imgs/print.png)

### Two methods to return printable representation of an object
![Str-repr](imgs/str_repr.png)

### Implementation: `str`

                class Customer:
                        def __init__(self, name, balance):
                                self.name, self.balance = name, balance

                        def __str__(self):
                                cust_str = """
                                Customer:
                                        name: {name}
                                        balance: {balance}
                                """.format(name = self.name, balance = self.balance)
                                return cust_str

                cust = Customer("Maryam", 30000)

                # Will implicitly call __str__()
                print(cust)

### Implementation: `repr`
- Surrond string arguments with quotation marks in `__repr__()` output

                class Customer:
                        def __init__(self, name, balance):
                                self.name, self.balance = name, balance

                        def __repr__(self):
                                # Notice the '...' around name
                                return "Customer('{name}', {balance})".format(name = self.name, balance = self.balance)

                cust = Customer("Maryam", 30000)
                cust # <--- Will implcitly call __repr__()


## Exceptions <a name="three"></a>
### Exception Handling
- Prevent the program from terminating when an exception is rise
- `try` - `except` - `finally`:

                try:
                        # Try running some code
                except ExceptionNameHere:
                        # Run this code if ExceptionNameHere happens
                except AnotherExceptionHere:       # <--- multiple except blocks
                        # Run this code if AnotherExceptionHere happens
                ...
                finally:                           # <--- optional
                        # Run this code no matter what

### Raising exceptions
- `raise ExceptionNameHere('Error message here')`

                def make_list_of_ones(length):
                        if length <= 0:
                                raise ValueError("Invalid length!")     # <--- Will stop the program and raise error
                        return [1]*length

### Exceptions are classes
- standard exceptions are inherited from `BaseException` or `Exception`

### Custom exceptions
- Inherit from `Exception` or one of its subclasses
- Usually an empty class

                class BalanceError(Exception):
                        pass

                class Customer:
                        def __init__(self, name, balance):
                                if balance < 0:
                                        raise BalanceError("Balance has to be non-negative!")
                                else:
                                        self.name, self.balance = name, balance

                cust = Customer("Larry Torres", -100)    # <--- Returns exception

- Exception interrupted the constructor $/rightarrow$ object not created

### Catching custom exceptions
  
                try:
                        cust = Customer("Larry Torres", -100)
                except BalanceError:
                        Customer("Larry Torres", 0)